In [ ]:
import dowhy as dw
from dowhy.causal_identifier import (
    AutoIdentifier,
    BackdoorAdjustment,
    IDIdentifier,
)

In [ ]:
from dowhy.causal_graph import CausalGraph
import dowhy.datasets

data = dowhy.datasets.linear_dataset(
    beta=4,
    num_common_causes=2,
    num_instruments=1,
    num_effect_modifiers=1,
    num_samples=500,
    treatment_is_binary=True,
    stddev_treatment_noise=10,
    num_discrete_common_causes=1,
)

In [ ]:
treatment_name = data["treatment_name"]
outcome_name = data["outcome_name"]

graph = CausalGraph(
    treatment_name=treatment_name,
    outcome_name=outcome_name,
    graph=data["gml_graph"],
    effect_modifier_names=data["effect_modifier_names"],
    common_cause_names=data["common_causes_names"],
    observed_node_names=data["df"].columns.tolist(),
)

# Functional API Usage

In [ ]:
# New functional API (AutoIdentifier)

identified_estimand_auto = dw.identify_effect(
    graph=graph,
    treatment=treatment_name,
    outcome=outcome_name,
    method=AutoIdentifier(estimand_type=dw.EstimandType.NONPARAMETRIC_ATE),
)
print(identified_estimand_auto)

In [ ]:
# New functional API (IDIdentifier)
identified_estimand_id = dw.identify_effect(
    graph=graph,
    treatment=treatment_name,
    outcome=outcome_name,
    method=IDIdentifier(estimand_type=dw.EstimandType.NONPARAMETRIC_ATE),
)
print(identified_estimand_id)

# Backwards compatibility

This section shows the same results as the new Functional API with the old `CausalModel` API

In [ ]:
# CausalModel API with AutoIdentifier

model = dw.CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"],
)

identified_estimand = model.identify_effect()
print(identified_estimand)

In [ ]:
# Another way of executing the identify effect by directly calling the object identify_effect method

identifier = AutoIdentifier(
    estimand_type=dw.EstimandType.NONPARAMETRIC_ATE, backdoor_adjustment=BackdoorAdjustment.BACKDOOR_DEFAULT
)

identified_estimand = identifier.identify_effect(
    graph=graph,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
)

print(identified_estimand)

In [ ]:
# CausalModel API with IDIdentifier 

model = dw.CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"],
)

identified_estimand = model.identify_effect(method_name="id-algorithm")
print(identified_estimand)

In [ ]:
# Another way of executing the identify effect by directly calling the object identify_effect method

identifier = IDIdentifier(estimand_type=dw.EstimandType.NONPARAMETRIC_ATE)

identified_estimand = identifier.identify_effect(
    graph=graph,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
)

print(identified_estimand)